In [1]:
from __future__ import print_function
import numpy as np
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from astropy.io import fits
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy import stats
import sys
import glob
import json
import seaborn as sns
import os.path
from collections import OrderedDict
from scipy.stats import gaussian_kde
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from astropy.table import Table
import pandas as pd
from itertools import combinations
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
import warnings
warnings.filterwarnings("ignore")

/home/luisangel/.conda/envs/luis-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 13:28:50.326768: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 13:28:50.327024: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 13:28:50.329328: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 13:28:50.357859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructi

In [48]:
# Read the ASCII file into an Astropy Table
data_gc = Table.read("catalogo_gcs_spectroscopio_splus_detected.dat", format='ascii')
data_nogc = Table.read("catalogo_contaminates_splus_detected.dat", format='ascii')
data_gc

NUMBER,ALPHA,DELTA,u,uerr,g,gerr,r,rerr,i,ierr,z,zerr,F378,F378err,F395,F395err,F410,F410err,F430,F430err,F515,F515err,F660,F660err,F861,F861err,fwhm_i,fwhm_psf_i,ellog_i,ellip_i,class_i,spread_i,flux_radius_i,area_i,a_image,b_image,theta_image,kron_radius
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
16253,52.65439772166957,-34.85515754646527,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,21.509762,0.4309228,99.0,99.0,99.0,99.0,20.985655,0.4293076,3.964676,1.36,1.2672191,0.21087044,0.6723346,-0.009377983,0.7654491,6,0.88541776,0.6987093,62.493454,3.5
16306,52.63961882667077,-34.85291674010414,99.0,99.0,21.505468,0.14491467,99.0,99.0,21.089676,0.12166906,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,21.286818,0.1148665,99.0,99.0,5.630442,1.36,1.0972694,0.08864683,0.06164774,0.007863585,2.1781638,11,0.9900843,0.90231645,77.92344,0.0
16324,52.65275307842115,-34.852162476132385,99.0,99.0,22.298355,0.28719342,21.956337,0.21723434,99.0,99.0,20.88752,0.37852666,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,5.430654,1.36,2.0048974,0.50122136,0.12759903,0.012198187,4.5430984,9,1.2915813,0.64421314,-70.62824,0.0
16364,52.63377244353862,-34.85072293012919,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,21.0796,0.3807831,99.0,99.0,99.0,99.0,3.4195957,1.36,1.7016889,0.4123485,0.48388717,0.00031423918,1.1321247,3,0.7585834,0.44578266,-19.666363,5.918421
16390,52.62834730730939,-34.849198337746614,99.0,99.0,22.041685,0.22105096,21.017162,0.08331954,20.51756,0.07139743,20.51217,0.12940818,99.0,99.0,99.0,99.0,18.82946,0.8450162,99.0,99.0,21.3907,0.27807727,20.945711,0.07998539,20.574175,0.15917358,3.7929711,1.36,1.2955253,0.22811234,0.88868934,0.0020403622,1.6645076,16,1.3120941,1.0127892,-70.39662,0.0
16397,52.626642977094306,-34.84923820813461,99.0,99.0,21.471018,0.48492202,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,2.2012887,1.36,1.2854649,0.22207135,0.49796444,0.0038801704,1.3151855,6,0.8411336,0.65434194,-44.52031,0.0
16458,52.632685126289566,-34.84701960314504,22.502518,0.5979565,99.0,99.0,22.013561,0.23860535,20.885118,0.42604688,21.18469,0.30696318,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,3.9009938,1.36,1.6480353,0.39321685,0.19945201,-0.0007584256,0.8174204,5,0.8476402,0.5143338,-44.52721,5.379644
16468,52.64617379576764,-34.846590308430436,99.0,99.0,22.934105,0.49898762,22.52174,0.34957176,21.769903,0.21682122,19.319366,1.503469,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,5.1380944,1.36,1.8970934,0.47287786,0.34734163,0.017193848,1.0781424,8,1.2750968,0.6721318,17.344378,0.0
16554,52.623203250535354,-34.84303056889895,21.649937,1.8276151,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,20.292194,1.781233,21.858763,0.47966313,99.0,99.0,99.0,99.0,3.4001122,1.36,1.641702,0.39087605,0.38951153,-0.00626068,2.1934,2,0.7410188,0.45137227,69.73203,6.697212


In [3]:
# convert in pandas
# Convert Astropy Tables to Pandas DataFrames
df_gc = data_gc.to_pandas()
df_nogc = data_nogc.to_pandas()

In [4]:
# Check for coincidences and remove rows in df_nogc that have matching 'NUMBER' in df_gc
df_nogc_filtered = df_nogc[~df_nogc['NUMBER'].isin(df_gc['NUMBER'])]

In [5]:
# Addding labels
# Add a new column named 'label' with value 0 to df1
df_gc['label'] = 0

# Add a new column named 'label' with value 1 to df2
df_nogc_filtered['label'] = 1

In [6]:
# Concatenate the two DataFrames vertically
combined_df = pd.concat([df_gc, df_nogc_filtered], ignore_index=True)
len(combined_df)

26523

#### Cleaning the data

In [7]:
# Cleaned the data error
m_err = (combined_df["rerr"] <= 0.2) & (combined_df["gerr"] <= 0.2) & \
        (combined_df["ierr"] <= 0.2) & \
      (combined_df["zerr"] <= 0.2)



df_cleanErr = combined_df[m_err]
len(df_cleanErr)

17303

In [8]:
# See how many object saty in each class
mask0 = df_cleanErr["label"] == 0
mask1 = df_cleanErr["label"] == 1
print("GC:", len(df_cleanErr[mask0]))
print("No GC:", len(df_cleanErr[mask1]))

GC: 177
No GC: 17126


## Preparing the data

In [9]:
#Selecting columns
columns = ["r",
           "g",
           "i",
           "z"]

In [10]:
df_mag = df_cleanErr[columns]
df_mag

,r,g,i,z
15,20.763142,21.272194,20.788673,20.632107
19,20.137680,21.452078,19.779497,19.552048
21,20.676428,21.163378,20.391083,20.389826
25,20.246616,21.061428,20.222399,20.175852
28,18.024874,18.803331,17.634941,17.343376
...,...,...,...,...
26518,14.743249,14.965096,14.748895,14.796816
26519,15.721187,16.303978,15.550023,15.479434
26520,16.063084,16.557777,15.940112,15.889799
26521,15.082020,15.565959,14.947059,14.911592


### Generating the colors

In [11]:
# Generate all combinations of magnitude columns
color_index_pairs = list(combinations(df_mag, 2))
len(color_index_pairs)

6

In [12]:
def calculate_earnings(df, index_pairs):
    for index_pair in index_pairs:
        color_index_name = f"{index_pair[0]} - {index_pair[1]}"
        df.loc[:, color_index_name] = df[index_pair[0]] - df[index_pair[1]]
    return df

In [13]:
df_colors_mag = calculate_earnings(df_mag, color_index_pairs)

In [14]:
df_colors_mag

,r,g,i,z,r - g,r - i,r - z,g - i,g - z,i - z
15,20.763142,21.272194,20.788673,20.632107,-0.509052,-0.025531,0.131035,0.483521,0.640087,0.156566
19,20.137680,21.452078,19.779497,19.552048,-1.314398,0.358183,0.585632,1.672581,1.900030,0.227449
21,20.676428,21.163378,20.391083,20.389826,-0.486950,0.285345,0.286602,0.772295,0.773552,0.001257
25,20.246616,21.061428,20.222399,20.175852,-0.814812,0.024217,0.070764,0.839029,0.885576,0.046547
28,18.024874,18.803331,17.634941,17.343376,-0.778457,0.389933,0.681498,1.168390,1.459955,0.291565
...,...,...,...,...,...,...,...,...,...,...
26518,14.743249,14.965096,14.748895,14.796816,-0.221847,-0.005646,-0.053567,0.216201,0.168280,-0.047921
26519,15.721187,16.303978,15.550023,15.479434,-0.582791,0.171164,0.241753,0.753955,0.824544,0.070589
26520,16.063084,16.557777,15.940112,15.889799,-0.494693,0.122972,0.173285,0.617665,0.667978,0.050313
26521,15.082020,15.565959,14.947059,14.911592,-0.483939,0.134961,0.170428,0.618900,0.654367,0.035467


In [15]:
# Drop magniytudes
df_colors = df_colors_mag.drop(columns=columns)
df_colors

,r - g,r - i,r - z,g - i,g - z,i - z
15,-0.509052,-0.025531,0.131035,0.483521,0.640087,0.156566
19,-1.314398,0.358183,0.585632,1.672581,1.900030,0.227449
21,-0.486950,0.285345,0.286602,0.772295,0.773552,0.001257
25,-0.814812,0.024217,0.070764,0.839029,0.885576,0.046547
28,-0.778457,0.389933,0.681498,1.168390,1.459955,0.291565
...,...,...,...,...,...,...
26518,-0.221847,-0.005646,-0.053567,0.216201,0.168280,-0.047921
26519,-0.582791,0.171164,0.241753,0.753955,0.824544,0.070589
26520,-0.494693,0.122972,0.173285,0.617665,0.667978,0.050313
26521,-0.483939,0.134961,0.170428,0.618900,0.654367,0.035467


In [16]:
# Labels
y = df_cleanErr['label']

## Appliying Random Forest

In [17]:
# Standardize the features (X) using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_colors)

In [18]:
# Increase the size of initial training set relative to X_pool
X_train, X_pool, y_train, y_pool = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

In [19]:
# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [20]:
# Initialize a random forest classifier with class weighting
classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

In [21]:
# Initialize ActiveLearner with uncertainty sampling
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_train_res, y_training=y_train_res,
    query_strategy=uncertainty_sampling,
)


In [22]:
# Define the number of iterations for active learning
iterations = 10  # Adjust as needed

In [23]:
# Active learning loop
for iteration in range(iterations):
    # Query the instances from the pool to be labeled using uncertainty_sampling
    query_idx, query_instance = learner.query(X_pool)

    # Debug statement to check query_idx
    print(f"Iteration {iteration + 1} - Queried indices: {query_idx}")

    # Check if there are enough instances to sample
    if query_idx.size == 0:
        print(f"Not enough instances to sample for iteration {iteration + 1}. Ending active learning.")
        break
    
    # Convert the queried index positions to integer indices
    query_idx = query_idx.flatten()
    
    # Check the length of query_idx to ensure it has the required number of instances
    if len(query_idx) < 5:  # Adjust the batch size as needed
        print(f"Not enough instances to sample for iteration {iteration + 1}. Ending active learning.")
        break

    # Simulate or perform manual labeling (replace with actual labeling process)
    labeled_idx = np.random.choice(query_idx, size=min(5, len(query_idx)), replace=False)
    X_label, y_label = X_pool[labeled_idx], y_pool[labeled_idx]

    # Teach the ActiveLearner with the newly labeled instances
    learner.teach(X=X_label, y=y_label)

    # Remove the newly labeled instances from the pool
    X_pool = np.delete(X_pool, labeled_idx, axis=0)
    y_pool = np.delete(y_pool, labeled_idx, axis=0)

    # Optionally, evaluate the model's performance after each iteration
    y_pred = learner.predict(X_scaled)  # Predict on entire dataset for evaluation
    print(f"Iteration {iteration + 1} - Classification Report:\n", classification_report(y, y_pred))
    print(f"Iteration {iteration + 1} - Confusion Matrix:\n", confusion_matrix(y, y_pred))

Iteration 1 - Queried indices: [294]
Not enough instances to sample for iteration 1. Ending active learning.


In [24]:
# Final evaluation after all iterations
final_pred = learner.predict(X_scaled)
print("Final Evaluation - Classification Report:\n", classification_report(y, final_pred))
print("Final Evaluation - Confusion Matrix:\n", confusion_matrix(y, final_pred))

Final Evaluation - Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.92      0.74       177
           1       1.00      0.99      1.00     17126

    accuracy                           0.99     17303
   macro avg       0.81      0.95      0.87     17303
weighted avg       1.00      0.99      0.99     17303

Final Evaluation - Confusion Matrix:
 [[  162    15]
 [   99 17027]]


Conclusion:

 Mejora en la Identificación de Globular Clusters mediante Técnicas de Aprendizaje Activo y Manejo del Desequilibrio de Clases

En este estudio, hemos demostrado cómo el uso de técnicas de aprendizaje activo, junto con estrategias para abordar el desequilibrio de clases, ha mejorado significativamente la capacidad de nuestro modelo para identificar Globular Clusters (GCs) en grandes conjuntos de datos astronómicos. A través del uso de SMOTE para el sobremuestreo de la clase minoritaria y la implementación de un clasificador basado en Random Forest con ponderación de clases, logramos mejorar el recall de los GCs, alcanzando un valor de 0.92 en nuestras evaluaciones finales.

Estos resultados no solo subrayan la eficacia de las técnicas empleadas para mitigar el desafío del desequilibrio de clases en la clasificación astronómica, sino que también resaltan la importancia de la precisión y el equilibrio en la detección de objetos astronómicos raros y significativos como los GCs. Estos hallazgos son cruciales para futuras investigaciones que buscan explorar y entender mejor la naturaleza y distribución de los GCs en el universo.

Nuestro enfoque no solo mejora la precisión de la identificación de GCs, sino que también proporciona una metodología robusta y reproducible que puede adaptarse a diferentes conjuntos de datos astronómicos y escenarios de investigación. Estos resultados respaldan la aplicación de técnicas avanzadas de aprendizaje automático en el campo de la astronomía para avanzar en nuestro entendimiento del universo y sus fenómenos más complejos.

## Apply to a big sample

In [25]:
# Example new sample data (X_sample) - Make sure X_sample is preprocessed similarly to X_train
data = Table.read("../Lomeli-data/catalog_all_bands_all_fovs_all_sources_106_rband.dat", format='ascii')

In [26]:

df_all = data.to_pandas()


In [27]:
df_all

,NUMBER,ALPHA,DELTA,u,uerr,g,gerr,r,rerr,i,...,fwhm_r,fwhm_psf_r,ellog_r,ellip_r,class_r,spread_r,flux_radius_r,area_r,mumax_r,kron_radius_r
0,5,41.707425,-31.609540,19.453903,0.102211,17.678207,0.008456,17.070164,0.005748,16.682482,...,28.233664,1.35,1.580351,0.367229,0.028626,0.024721,10.207849,1979,18.582102,3.500000
1,6,42.169686,-31.536274,20.278736,0.066860,18.145863,0.010459,17.325224,0.006315,16.626760,...,35.868310,1.35,5.854529,0.829192,0.028549,0.016807,15.456670,2995,19.180944,3.500000
2,9,41.837251,-31.610296,16.722420,0.006870,14.648617,0.001814,13.828032,0.001178,13.586303,...,3.187606,1.35,1.257020,0.204468,0.999676,0.000744,1.706050,534,15.280809,3.500000
3,10,42.123689,-31.610100,99.000000,99.000000,17.397250,0.007173,17.111893,0.005796,17.074059,...,15.544360,1.35,2.551127,0.608016,0.028643,0.030338,4.576429,684,18.362354,3.500000
4,11,41.981585,-31.606249,21.427402,0.186251,19.980090,0.036666,19.361853,0.022071,19.016304,...,11.915299,1.35,1.037807,0.036430,0.000408,0.025243,6.217256,362,21.232092,3.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749399,31975,58.172880,-40.790009,21.073397,0.153950,19.439800,0.038556,18.584753,0.019846,18.299133,...,2.829213,1.46,1.049959,0.047582,0.999702,0.000744,1.639664,53,19.885939,3.500000
1749400,31976,56.880399,-40.744566,99.000000,99.000000,21.011417,0.114948,20.388622,0.061590,20.034092,...,14.759645,1.46,1.708737,0.414772,0.000335,0.019123,9.469728,76,22.024738,6.373395
1749401,31979,57.352097,-40.795468,99.000000,99.000000,99.000000,99.000000,21.529171,0.152961,21.147512,...,8.299409,1.46,1.246947,0.198041,0.008161,0.009048,2.399784,17,22.620626,3.500000
1749402,31980,57.672448,-40.797118,99.000000,99.000000,99.000000,99.000000,22.054295,0.249689,21.836752,...,5.348808,1.46,1.512234,0.338726,0.478432,-0.008490,1.922813,4,22.879822,5.111665


In [28]:
# Cleaned the data
m_x =  (df_all["r"] >= 13) & (df_all["r"] <= 23)
m_err = (df_all["rerr"] <= 0.2) & (df_all["gerr"] <= 0.2) & \
         (df_all["ierr"] <= 0.2) & (df_all["zerr"] <= 0.2)

flags =  (df_all["flags_i"] == 0)

mask = m_x & m_err & flags

df_all_clean = df_all[mask]
len(df_all_clean)

740978

In [29]:
df_mag_all = df_all_clean[columns]
df_mag_all

,r,g,i,z
0,17.070164,17.678207,16.682482,16.366240
2,13.828032,14.648617,13.586303,13.472961
3,17.111893,17.397250,17.074059,16.979103
4,19.361853,19.980090,19.016304,18.795696
5,17.221579,18.326470,16.830046,16.648365
...,...,...,...,...
1749385,20.024786,21.375532,19.633955,19.323668
1749389,20.698190,21.496752,20.548940,20.467710
1749394,19.572569,20.717197,19.126050,18.891996
1749398,20.682095,21.103998,20.406397,20.181276


In [30]:
# Making the colors
df_colors_mag_all = calculate_earnings(df_mag_all, color_index_pairs)

In [31]:
df_colors_mag_all

,r,g,i,z,r - g,r - i,r - z,g - i,g - z,i - z
0,17.070164,17.678207,16.682482,16.366240,-0.608043,0.387682,0.703924,0.995725,1.311967,0.316242
2,13.828032,14.648617,13.586303,13.472961,-0.820585,0.241729,0.355071,1.062314,1.175656,0.113342
3,17.111893,17.397250,17.074059,16.979103,-0.285357,0.037834,0.132790,0.323191,0.418147,0.094956
4,19.361853,19.980090,19.016304,18.795696,-0.618237,0.345549,0.566157,0.963786,1.184394,0.220608
5,17.221579,18.326470,16.830046,16.648365,-1.104891,0.391533,0.573214,1.496424,1.678105,0.181681
...,...,...,...,...,...,...,...,...,...,...
1749385,20.024786,21.375532,19.633955,19.323668,-1.350746,0.390831,0.701118,1.741577,2.051864,0.310287
1749389,20.698190,21.496752,20.548940,20.467710,-0.798562,0.149250,0.230480,0.947812,1.029042,0.081230
1749394,19.572569,20.717197,19.126050,18.891996,-1.144628,0.446519,0.680573,1.591147,1.825201,0.234054
1749398,20.682095,21.103998,20.406397,20.181276,-0.421903,0.275698,0.500819,0.697601,0.922722,0.225121


In [32]:
# Drop magnitudes
df_colors_all = df_colors_mag_all.drop(columns=columns)
df_colors_all

,r - g,r - i,r - z,g - i,g - z,i - z
0,-0.608043,0.387682,0.703924,0.995725,1.311967,0.316242
2,-0.820585,0.241729,0.355071,1.062314,1.175656,0.113342
3,-0.285357,0.037834,0.132790,0.323191,0.418147,0.094956
4,-0.618237,0.345549,0.566157,0.963786,1.184394,0.220608
5,-1.104891,0.391533,0.573214,1.496424,1.678105,0.181681
...,...,...,...,...,...,...
1749385,-1.350746,0.390831,0.701118,1.741577,2.051864,0.310287
1749389,-0.798562,0.149250,0.230480,0.947812,1.029042,0.081230
1749394,-1.144628,0.446519,0.680573,1.591147,1.825201,0.234054
1749398,-0.421903,0.275698,0.500819,0.697601,0.922722,0.225121


In [33]:
# Ensure X_sample is transformed using the same scaler as X_train
X_sample_scaled = scaler.transform(df_colors_all)

In [34]:
# Predict with the trained model
y_pred_sample = learner.predict(X_sample_scaled)

In [35]:
# Predict probabilities with the trained model
y_prob_sample = learner.predict_proba(X_sample_scaled)

In [36]:
# Print predicted labels for sample data
print("Predicted Labels for Sample Data:\n", y_pred_sample)
# Print predicted probabilities for sample data
print("Predicted Probabilities for Sample Data:\n", y_prob_sample)

# Optionally, evaluate predictions on sample data if ground truth labels are available
# y_true_sample = ...  # Ground truth labels if available
# print("Evaluation on Sample Data - Classification Report:\n", classification_report(y_true_sample, y_pred_sample))
# print("Evaluation on Sample Data - Confusion Matrix:\n", confusion_matrix(y_true_sample, y_pred_sample))


Predicted Labels for Sample Data:
 [1 1 1 ... 1 1 1]
Predicted Probabilities for Sample Data:
 [[0.09 0.91]
 [0.02 0.98]
 [0.   1.  ]
 ...
 [0.01 0.99]
 [0.48 0.52]
 [0.31 0.69]]


In [37]:
# Count number of objects with label 0 and 1 in y_pred_sample
count_label_0 = np.count_nonzero(y_pred_sample == 0)
count_label_1 = np.count_nonzero(y_pred_sample == 1)

# Print the counts
print(f"Number of objects labeled as 0 (GC): {count_label_0}")
print(f"Number of objects labeled as 1 (Non-GC): {count_label_1}")

Number of objects labeled as 0 (GC): 61032
Number of objects labeled as 1 (Non-GC): 679946


In [38]:
df_all_clean["Label"] = y_pred_sample

In [39]:
df_all_clean['Prob(GC)'] = y_prob_sample[:,0]
df_all_clean['Prob(Non-CG)'] = y_prob_sample[:,1]

In [40]:
# Example usage after adding columns
print(df_all_clean[['Label', 'Prob(GC)', 'Prob(Non-CG)']].head())  # Print the first few rows for verification

   Label  Prob(GC)  Prob(Non-CG)
0      1      0.09          0.91
2      1      0.02          0.98
3      1      0.00          1.00
4      1      0.00          1.00
5      1      0.00          1.00


In [47]:
mask_cg = df_all_clean['Prob(GC)'] > 0.95
print("High probility GC:", len(df_all_clean[mask_cg]))

High probility GC: 7195


In [45]:
# Step 1: Filter the DataFrame for GC classified instances (Label == 0)
df_gc_only = df_all_clean[df_all_clean['Label'] == 0]

# Step 2: Save the filtered DataFrame to a CSV file
df_gc_only.to_csv('predicted_GC_results_only_broadFilters-smote.csv', index=False)  # Adjust the filename as needed
